### [HDFS论文](https://static.googleusercontent.com/media/research.google.com/zh-CN//archive/gfs-sosp2003.pdf)

### 一. HDFS文件系统的假设和目标
#### 1. 硬件失败是正常的
* hdfs认为Hardware Failure属于正常情况而非一个exeption; 一些组件会偶尔失效
* 因此,hdfs的目标是能发现这些硬件错误, 并自动快速的从中恢复过来

#### 2. 流式文件访问   
* HDFS是为批处理提供的文件系统, 而不是为了提供一个与用户交互的文件系统; 因此, HDFS的目标是高数据吞吐,不是低数据访问延迟

#### 3. 简单一致性模式
* HDFS上的文件一次写多次读. 文件已经创建完毕, 不能再被修改, 只能在文件末尾追加内容

#### 4. Moving Computation is Cheaper than Moving Data
* 把计算的执行运行在离数据存储的机器较近的节点, 比移动数据要更加高效.
* HDFS提供了这样的接口, 把应用发往数据存储较近的节点

### 二. HDFS架构

#### 1. NameNode和DataNode
* HDFS是master/slave架构.namenode是master,datanode是slave
* NameNode和DataNode的文件职责
    * NameNode: 完成文件系统命名空间(文件树)的操作, 比如:打开,关闭,重命名文件/文件夹 
    * DataNode: 作为处理读写的服务端, 返回给客户端response.
        主要完成读,写,block的创建,删除,复制副本
* NameNode
    * NameNode是仲裁者的角色, 并且是hdfs所有原数据的仓库
    * HDFS文件的数据从来不经过NameNode
    * 文件系统命名空间(文件树)的任何变化都会被记录在Namenode中

#### 2. 数据副本



#### 2. hdfs文件系统对比磁盘文件系统
* 文件拆接到不同扇区放到应哦按上
* 文件拆解成不同Block放到不同机器上

#### 3. 免密码登录的作用:   
NameNode可以通过ssh到datanode(slave文件中配置的), 执行hadoop-deamon.sh start datanode启动DataNode
    
#### 4. haddop一直处于safe mode的问题?  
 block汇报数量达不到namenode的要求, 说明有block丢失了
 
#### 5. 什么是safe mode?    
 * Datanode启动后, 需要向namenode汇报自己持有的block数, 和namenode自己记录的总block数比较, 达到0.9999后才可以启动集群
 * checkpoint   
    * hdfs上的操作记录日志editlog, 到达一个checkpoint后, 合并fsimage与editlog形成新的fsimage
    * secondary namenode原本是为了合并fsimage和editlog,   
    * 现在由standby namenode做这部分工作, 而且作为另一个热备
 
#### 6. fsimage中记录的什么  
记录的文件目录树

#### 7. 共享存储 jornal node (paxos实现)
* jornal node只记录
* 什么算create成功?
* 先写namenode, 再写jornalnode
    
#### 8. federation
* federation的作用, 把整个文件系统的文件目录树劈开, 分给不同的namenode管理
* 所有的datanode向所有的namenode汇报, 但是datanode只把属于某个namenode的block信息汇报给该namenode


#### 9. Block
只有namenode知道文件到block的映射
datanode不知道这个block属于哪一个文件, datanode写的是名为"blk_blockid"的文件